# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Подготовка данных

In [84]:
import pandas as pd
import numpy as np

data = pd.read_csv('data.csv')


data.info() #общая информация о таблице

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [85]:
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [86]:
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [87]:
data['family_status_id'].value_counts()

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

Все виды семейного статуса соответствуют своему type_id

In [88]:
data.sort_values(by='days_employed', ascending=True) #визуально оцениваем данные о количестве отработанных дней

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
16335,1,-18388.949901,61,среднее,1,женат / замужем,0,F,сотрудник,0,186178.934089,операции с недвижимостью
4299,0,-17615.563266,61,среднее,1,женат / замужем,0,F,компаньон,0,122560.741753,покупка жилья
7329,0,-16593.472817,60,высшее,0,женат / замужем,0,F,сотрудник,0,124697.846781,заняться высшим образованием
17838,0,-16264.699501,59,среднее,1,женат / замужем,0,F,сотрудник,0,51238.967133,на покупку автомобиля
16825,0,-16119.687737,64,среднее,1,женат / замужем,0,F,сотрудник,0,91527.685995,покупка жилой недвижимости
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


**Вывод**

Заметки:
1) В столбец days_employed очень проблемный выгружены данные с отрицательным значениям в днях, природа ошибки не понятна, так как имеются значения  со знаками плюс и минус. Так же в столбце имеются имеются нереалистичные 6-ти значные. Имеются прпущенные значения обозначающие что человек является безработным

2) Значения в сталбцах family_status и family_status_id соответствуют друг другу.
3) Столбец education необходимо привести к общему виду (low case)
4) Стобец dob_years (возраст клиента) имеет нулевые значения что так же не является отражением реальности

## Предобработка данных

### Обработка пропусков

С помощью применения связки методов isnull().any(axis=1) можем получить уникальную таблицу со всеми строками содержащими пропуски.

Логика работы следующая: создаём логическую серию и использовать ее для индексации нашего датафрейма:

In [89]:
data[data.isnull().any(axis=1)] # отображение всех строчек содержащих NaN

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


### Работа с NaN

Заменим значения NaN в колонках days_employed и total_income на 0, т.к. отсутствие значений говорит нам о нулевом рабочем стаже заёмщика, а соответственно и о нулевом доходе. Количество отсутствующих записей совпадают

В дальнейшем значения нулевые значения в total_income будут заменены на медиану для каждой категории граждан, которые будут выделены позже

In [90]:
data['days_employed'].isnull().sum() # проверяем соответствие количество 
                                     # данных в обеих колонках, чтобы убедиться во взаимосвязи

2174

In [91]:
data['total_income'].isnull().sum()

2174

In [92]:
data['days_employed'] = data['days_employed'].fillna(0) # заменяем NaN на 0

In [93]:
data[data.isnull().any(axis=1)].sort_values('dob_years', ascending=True) #проверяем -> ждём пустую таюлицу

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12403,3,0.0,0,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,операции с коммерческой недвижимостью
6670,0,0.0,0,Высшее,0,в разводе,3,F,пенсионер,0,NaN,покупка жилой недвижимости
2284,0,0.0,0,среднее,1,вдовец / вдова,2,F,пенсионер,0,NaN,недвижимость
6411,0,0.0,0,высшее,0,гражданский брак,1,F,пенсионер,0,NaN,свадьба
19829,0,0.0,0,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,жилье
...,...,...,...,...,...,...,...,...,...,...,...,...
8456,0,0.0,71,среднее,1,женат / замужем,0,M,пенсионер,0,NaN,операции с коммерческой недвижимостью
11548,0,0.0,71,среднее,1,женат / замужем,0,M,пенсионер,0,NaN,покупка жилой недвижимости
13864,0,0.0,72,среднее,1,женат / замужем,0,F,компаньон,0,NaN,сделка с автомобилем
10563,0,0.0,72,Среднее,1,Не женат / не замужем,4,F,пенсионер,1,NaN,приобретение автомобиля


Для большей правдивости данных заполним пустые значения в колонке total_income медианой для каждой из возрастных групп граждан

In [94]:
total_income_median = data.loc[: , ['education_id', 'income_type', 'total_income']] # выделил необходимые столбцы
age_groups = [[0, 20], [21, 25] ,[26, 30], [31, 35], [36, 40], [41, 45], 
              [46, 50], [51, 55], [56, 60], [61, 65], [66, 70], [71, 74]] # подготовил возрастные категории

In [95]:
total_income_median.groupby('education_id')['total_income'].mean()

education_id
0    207142.515219
1    153715.643971
2    181534.022774
3    132155.513626
4    174750.155792
Name: total_income, dtype: float64

In [97]:
df = data.copy()
df.education = df.education.str.lower()

df.pivot_table(index=['income_type', 'gender'], columns='education', values='total_income', aggfunc='median')

education                      высшее      начальное  неоконченное высшее  \
income_type     gender                                                      
безработный     F       202722.511368            NaN                  NaN   
                M                 NaN            NaN                  NaN   
в декрете       F                 NaN            NaN                  NaN   
госслужащий     F       157841.438279   89618.962330        136599.129610   
                M       214706.640963  190966.659534        177320.628254   
компаньон       F       189893.464621  134012.003567        170471.622782   
                M       233064.883613  150100.960964        191291.929274   
                XNA               NaN            NaN        203905.157261   
пенсионер       F       142404.278858  101180.566458        116592.976550   
                M       157457.331331  113124.202781        124667.471301   
предприниматель F       499163.144947            NaN                  NaN   
сотрудник       F       153691.851098  118516.720741        134925.188145   
                M       192247.298639  141637.960402        189075.918446   
студент         M        98201.625314            NaN                  NaN   

education                     среднее  ученая степень  
income_type     gender                                 
безработный     F                 NaN             NaN  
                M        59956.991984             NaN  
в декрете       F        53829.130729             NaN  
госслужащий     F       125831.806597   111392.231107  
                M       169651.956756             NaN  
компаньон       F       144881.784198             NaN  
                M       181889.020944             NaN  
                XNA               NaN             NaN  
пенсионер       F       112790.975121   255425.196556  
                M       126097.196750    98752.495442  
предприниматель F                 NaN             NaN  
сотрудник       F       124127.090016             NaN  
                M       156286.402929   198570.757322  
студент         M                 NaN             NaN

In [98]:
def super_fillna_func(income_type, gender, education):
    '''
    Находит в таблице qq нужную медиану.
    '''
    try:
        return qq.loc[(income_type, gender)][education]
    except:
        return 0
    
print(super_fillna_func('студент', 'M','высшее') ) 

98201.62531401133


In [99]:
df = df.apply(lambda row: super_fillna_func(row['income_type'], row['gender'], row['education']), axis=1)

In [100]:
index_income = data.loc[data['total_income'].isnull() == True]['total_income'].index

In [101]:
data.loc[index_income, 'total_income'] = df[index_income]

In [102]:
data.sort_values(by='total_income') # визуальная оценка только что заполненных данных

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
5936,0,0.000000,58,высшее,0,женат / замужем,0,M,предприниматель,0,0.000000e+00,покупка жилой недвижимости
14585,0,359219.059341,57,среднее,1,женат / замужем,0,F,пенсионер,1,2.066726e+04,недвижимость
13006,0,369708.589113,37,среднее,1,гражданский брак,1,M,пенсионер,0,2.120528e+04,заняться высшим образованием
16174,1,-3642.820023,52,Среднее,1,женат / замужем,0,M,сотрудник,0,2.136765e+04,приобретение автомобиля
1598,0,359726.104207,68,среднее,1,гражданский брак,1,M,пенсионер,0,2.169510e+04,на проведение свадьбы
...,...,...,...,...,...,...,...,...,...,...,...,...
17178,0,-5734.127087,42,высшее,0,гражданский брак,1,M,компаньон,0,1.711309e+06,сыграть свадьбу
20809,0,-4719.273476,61,среднее,1,Не женат / не замужем,4,F,сотрудник,0,1.715018e+06,покупка жилья для семьи
9169,1,-5248.554336,35,среднее,1,гражданский брак,1,M,сотрудник,0,1.726276e+06,дополнительное образование
19606,1,-2577.664662,39,высшее,0,женат / замужем,0,M,компаньон,1,2.200852e+06,строительство недвижимости


**Вывод**

В данных отсутствовали 2174 значения о стаже работы и месячном доходе. Природа их появления слнедующая:  
1) Клиенты действительно не работают и система автоматически пропускает значения total_income(месчного дохода) у безработных клиентов, т.к. кореляция между этими значениями 100%

2) Клиенты не указывают место работы исходя из личных сообржений

Данные ошибки в данных появились, вероятно, из-за смены формата записи их в базу данных

Более связей по двум столбцам не найдено

3) Имеется значение 0 в колонке dob_years(возраста клиентов), но т.к. это не имеет значение к поставленной задаче и принято решение оставить его нетронутым. Причина появляения - не указан возраст при подаче заявки

Данная ошибка могла появиться из-за человеческого фактора, у клиентов была возможность не указывать свой возраст или же форма заполнения можно было перепутать с графой "дата подачи заявки". Нужно оповестить разработчиков что вероятнее всего или UX дизайнеров



### Замена типа данных

План:

1) Испрвить значения стажа на абсолютные значения

2) Привести столбец education к общему виду

3) Перевести вещественные знаения в целочисленные

4) Привести огромные значения в колонке days_employed к соответствующему виду, т.е. стаж в количество рабочих дней. Основная гипотеза - ошибка возникает потому что время приведено в часах, а не сутках.



In [103]:
# замена на абсолютные числа

data['days_employed'] = abs(data['days_employed']) 

In [104]:
# перевод столбца education к общему виду

data['education'] = data['education'].str.lower() 

In [105]:
data.head(10) # контроль

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Приводим данные по колонки days_employed к коректным значениям, для этого я определил методом pd.df.describe() что огромный скачёк в значениях происходит в 4-м квантиле отсортированных по возрастанию значений колонки days_employed. это дало мне основание для того, чтобы выбрать значение 20000 как то, которое гарантированно будет отсукать корректные данные от некоректных.
Далее, исходя из гипотезы что некоректные данные это часы, а не сутки я разделил значения на 24, чтобы часы соответствовали дням.

In [106]:
# Перевод некоректных значений рабочего стажа из часов в дни
data.loc[data['days_employed'] > 20000, 'days_employed'] = data.loc[data['days_employed'] > 20000, 'days_employed'] / 24

Приводим вещественные колонки к целочисленному виду. Для изменения типа данных был выбран стандартный метод astype() т.к. метод to_numeric() приводит данные к типу floan64

In [107]:
# смена типа данных

data[['days_employed', 'total_income']] = data[['days_employed', 'total_income']].astype('int')

In [108]:
data.head(10) # визуальная проверка данных

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


**Вывод**

В данных был ряд проблем: некоректные и отрицательные значения, разный формат заполнения строк в ячейках. Все проблемы были устранены и приведены в удобный для обработки вид

### Обработка дубликатов

In [109]:
data.duplicated().sum() # Определяем количество явных дупликатов

71

In [110]:
data = data.drop_duplicates().reset_index(drop=True) # убираем все явные дупликаты

In [111]:
data.duplicated().sum() # проверяем количество дубликатов после чистки

0

**Вывод**

Дупликаты в данных могли появится по причинам: случайного совпадения, технической ошибки или рукописного ввода во время принятии заявки(человеческого фактора).

Наша задача определить как один набор уникальных значений X влияет на конечный результат Y, это говорит нам о том, что дублирования одних и тех же наборов данных нам не интересно, так как результат этих данных уже будет известен. Поэтому мы можем свободно избавиться от дупликатов в данных. Метод сработал удачно и можем приступать к следующему этапу

### Лемматизация

In [112]:
import sys # импортируем библиотеку sys для возможности установки сторонних библиотек
!{sys.executable} -m pip install pymystem3

In [113]:
from pymystem3 import Mystem # импортируем pymystem3
from collections import Counter # импортируем Counter для работы с лемматизацией
m = Mystem() #назначаем переменную для комфортного взаимодействия с функцей


In [114]:
list_to_text = ' '.join(map(str, data['purpose'])) # с помощью функции map применяем метод str к пандасовской серии, 
                                                         # перевод каждого элемента в строку
                                                         # join к каждому элементу в списке

lemmas = m.lemmatize(list_to_text) #применяем лемматизацию

print(Counter(lemmas)) # видим результат используя метод Counter для наглядности

Counter({' ': 55023, 'недвижимость': 6351, 'покупка': 5897, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2324, 'свой': 2230, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'подержать': 853, 'проведение': 768, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'приобретение': 461, 'профильный': 436, 'подержанный': 111, '\n': 1})


**Вывод**

Для работы в локальной версии нужно было импортировать стандартную библиотеку пайтона для возможности загружать нестандартные библиотеки, какой и является pymystem3.

После импорта всех необходимых библиотек и назначения технической переменной 'm' пришло время обрабатывать данные.
Для начала нужно было перевести пандасовскую Серию в единую строку для последующей лемматизации.

Применив к полученным данным метод Counter мы видим количества каждого применимого слова, это делает данные гораздо более однозначными. 

Людей больше всего интересует покупка недвижимости, второе место заняло кредитования для покупки автомобиля и на третьем месте получение образования, на четвёртом проведение свадьбы

### Категоризация данных

Для категоризации данных по цели покупки нам необходимо будет применить стемминг используя библиотеку nltk

In [115]:
from nltk.stem import SnowballStemmer # импортируем из библиотеки nltk.stem нужный нам модуль
nl = SnowballStemmer('russian') # назначаем тихническую переменную для работы с русским текстом

In [116]:
list_of_purpose = list(data['purpose'].unique()) #Выделим все уникальные значения целей покупки

Теперь нам необходимо вычленить все категории. На первый взгляд их будет несколько: недвижимость, автомобиль, образование, здоровье, свадьба. Сейчас проверим это с помощью стемминга.

In [117]:
stemmer_list = [] #пустой список для значений стемминга

for item in list_of_purpose: # профодим стемминг по списку целей покупки
        
    for word in item.split(" "):
        stemmer_list.append(nl.stem(word))
        
        
sorted_stemmer_list = sorted(set(stemmer_list))
print(sorted_stemmer_list)

['автомоб', 'автомобил', 'высш', 'для', 'дополнительн', 'жил', 'заня', 'коммерческ', 'на', 'недвижим', 'образован', 'операц', 'подержа', 'покупк', 'получен', 'приобретен', 'проведен', 'профильн', 'ремонт', 'с', 'свадьб', 'сво', 'сдач', 'сделк', 'сем', 'со', 'собствен', 'строительств', 'сыгра']


In [118]:
data['purpose_category'] = np.nan # создаём пустую колонку для дальнейшего заполнения.

Был выбрат способ np.nan, который заполняет весь столбец отсутствующими значениями. Был выбран этот метод по причине того, что на данном этапе у нас нет данных которыми мы готовы заполнить столбец


Так как при лемминге гипотеза подтвердилась у нас имеются 4 освновных категории, вычленим уникальные значения из колонки data['purpose'] из категорий

In [120]:
def categorization(row):
    
    stemmed_query = nl.stem(row)
    
    for word in stemmed_query.split(" "):
        
        if (word == 'жил') or (word == 'недвижим')or (word == 'жилья'):
            return 'недвижимость'
            
        elif (word == 'автомобил') or (word == 'автомоб'):
             return 'автомобиль'
            
        elif word == 'образован':
            return 'образование'
            
        elif word == 'свадьб':
            return 'свадьба'



In [121]:
data['purpose_category'] = data['purpose'].apply(categorization) # применяем метод эплай к каждой строке столбца 'prupose'

In [122]:
data.loc[:, ['purpose', 'purpose_category']].head(5) # срез для контроля

,purpose,purpose_category
0,покупка жилья,недвижимость
1,приобретение автомобиля,автомобиль
2,покупка жилья,недвижимость
3,дополнительное образование,образование
4,сыграть свадьбу,свадьба


Далее нужно будет нарисовать таблицу в которой бы столбцы с целью были заменены на 4 категории

In [123]:
data.drop(labels='purpose', axis=1) # убираем лишний столбец, т.к. на его основе уже проведена категоризация

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба
...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,недвижимость
21450,0,14330,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,автомобиль
21451,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21452,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,автомобиль


In [124]:
data['purpose_category'].unique() # проверяем коректно ли сработал код для присвоения категорий и нет ли там лишних значений

array(['недвижимость', 'автомобиль', 'образование', 'свадьба'],
      dtype=object)

**Вывод**

Стемминговое исследование выделило 4 основных категории: недвижимость, автомобиль, образование, свадьба. Для выделения категории была составлена отдельная колонка data['purpose_category'] и заполнена в соответствии с клиентским запросом

In [125]:
children_debt_dict=data[['children','debt', 'family_status_id']] # выделим необходимые столбцы для анализа

В колонке ['children'] имеются отрицательные (-1 детей) и неадекватные значения (20 детей)

Т.к. в столбце children есть отрицательные значения нужно использовать метод abs(). Этот метод вернёт абсолютное заначения каждого компонента столбца

In [126]:
children_debt_dict['children'] = children_debt_dict['children'].abs() # избавляемся от отрицательных значений

<ipython-input-126-7b980c4dabdb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  children_debt_dict['children'] = children_debt_dict['children'].abs() # избавляемся от отрицательных значений


In [127]:
children_debt_dict.loc[children_debt_dict['children'] == 20] = 2 # заменим значение 20

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [128]:
children_debt_results = pd.pivot_table(children_debt_dict, index=["children"], columns='debt', values='family_status_id', aggfunc='count')

In [129]:
children_debt_results

debt,0,1,2
children,,,
0,13028.0,1063.0,NaN
1,4410.0,445.0,NaN
2,1858.0,194.0,76.0
3,303.0,27.0,NaN
4,37.0,4.0,NaN
5,9.0,NaN,NaN


In [130]:
children_debt_results.loc[0 , 1] # Первое число - количество детей, второе - есть/нет задолжености

1063.0

In [131]:
children_debt_results.loc[0].sum() # сумма конкретной категории

14091.0

In [151]:
def my_mean(x): return '{:.2%} '.format(x.mean())

data.groupby('children')['debt'].agg(['count', 'sum', my_mean])

,count,sum,my_mean
children,,,
-1,47,1,2.13%
0,14091,1063,7.54%
1,4808,444,9.23%
2,2052,194,9.45%
3,330,27,8.18%
4,41,4,9.76%
5,9,0,0.00%
20,76,8,10.53%


**Вывод**

Я привёл данные тремя разными способами: сортировка датафрейма, группировка и сводная таблица. В данном случае самым удобным считаю метод groupby() за счёт длины кода и информативности.

Полученные результаты говорят что самый высокий процент заделженности среди клиентов у тех, кто имеет по 20 детей, хотя скорее всего это ошибка в данных, но оставим как есть, следуя условиям задания, а самый низкий процент у клиентов имеющих 5 детей в семье.

В среднем процент задолженности среди всех клиентов составляет 10% и не корелируется с количеством детей в семье

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [154]:
data.groupby('family_status')['debt'].agg(['count', 'sum', my_mean])

,count,sum,my_mean
family_status,,,
Не женат / не замужем,2810,274,9.75%
в разводе,1195,85,7.11%
вдовец / вдова,959,63,6.57%
гражданский брак,4151,388,9.35%
женат / замужем,12339,931,7.55%


**Вывод**

Привёл данные тремя разными способами: сортировка датафрейма, группировка и сводная таблица. В данном случае самым удобным считаю метод groupby() за счёт длины кода и информативности.

По данным можно сделать интересные выводы о том, не женатые/не замужные и состоящие в гражданском браке клиенты имеют больший процент задолженности чем остальные социальные группы

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Чтобы коректно определить уровни дохода предлогаю использовать разделение на 3 класса, низший от 0 до 12792 (МРОТ), средний от 12792 до 120000 и высший от 120000 рублей

In [140]:
total_income_debt_dict=data[['total_income','debt', 'family_status_id']] # выделим необходимые столбцы для анализа
total_income_debt_dict['total_income_category'] = np.nan # создаём пустой столбец для дальнейшего присвоения категории

<ipython-input-140-dc7ca454ce97>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_income_debt_dict['total_income_category'] = np.nan # создаём пустой столбец для дальнейшего присвоения категории


In [141]:
data['total_income'].sort_values().describe()

count    2.145400e+04
mean     1.654622e+05
std      9.840914e+04
min      0.000000e+00
25%      1.074990e+05
50%      1.448810e+05
75%      1.968970e+05
max      2.265604e+06
Name: total_income, dtype: float64

In [157]:
for item in range(0, 21454): # цикл заполняющий столбец total_income_category относительно предложенных критериев
    
    if total_income_debt_dict['total_income'][item] >= 0 and \
       total_income_debt_dict['total_income'][item] < 1.076230e+05:
        total_income_debt_dict['total_income_category'][item] = 'низкий'
                  
    elif total_income_debt_dict['total_income'][item] >= 1.076230e+05 and total_income_debt_dict['total_income'][item] <= 1.472920e+05:
        total_income_debt_dict['total_income_category'][item] = 'средний'
        
    elif total_income_debt_dict['total_income'][item] > 1.472920e+05 and total_income_debt_dict['total_income'][item] <= 1.958132e+05:
        total_income_debt_dict['total_income_category'][item] = 'высокий'
        
    elif total_income_debt_dict['total_income'][item] > 1.958132e+05 and total_income_debt_dict['total_income'][item] <= 2.265604e+06:
        total_income_debt_dict['total_income_category'][item] = 'высший'

<ipython-input-157-ebd7b0bccdd9>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_income_debt_dict['total_income_category'][item] = 'высший'
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
<ipython-input-157-ebd7b0bccdd9>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_income_debt_dict['total_i

In [158]:
total_income_debt_dict # проверяем коректность заполнения

,total_income,debt,family_status_id,total_income_category
0,253875,0,0,высший
1,112080,0,0,средний
2,145885,0,0,средний
3,267628,0,0,высший
4,158616,0,1,высокий
...,...,...,...,...
21449,224791,0,1,высший
21450,155999,0,0,высокий
21451,89672,1,1,низкий
21452,244093,1,0,высший


In [168]:
total_income_debt_result = pd.pivot_table(total_income_debt_dict, index=["total_income_category"], columns='debt', values='family_status_id', aggfunc=['count'])# группируем данные для анализа

In [174]:
total_income_debt_result

count     
debt                      0    1
total_income_category           
высокий                4509  436
высший                 5055  386
низкий                 4945  427
средний                5204  492

In [183]:
print('процент клиентов высокого класса имеющих задолженость: {0:.2%}'.format((total_income_debt_result.loc['высокий'][1] / 
                                                                              total_income_debt_result.loc['высокий'].sum())))
print('процент клиентов высшего класса имеющих задолженость: {0:.2%}'.format(total_income_debt_result.loc['высший'][1] / 
                                                                             total_income_debt_result.loc['высший'].sum()))
print('процент клиентов низкого класса имеющих задолженость: {0:.2%}'.format((total_income_debt_result.loc['низкий'][1] / 
                                                                              total_income_debt_result.loc['низкий'].sum())))
print('процент клиентов среднего класса имеющих задолженость: {0:.2%}'.format((total_income_debt_result.loc['средний'][1] / 
                                                                             total_income_debt_result.loc['средний'].sum())))

процент клиентов высокого класса имеющих задолженость: 8.82%
процент клиентов высшего класса имеющих задолженость: 7.09%
процент клиентов низкого класса имеющих задолженость: 7.95%
процент клиентов среднего класса имеющих задолженость: 8.64%


**Вывод**

Во время предобработки я заместил все отсутствующие значения в графе total_income на медианы для каждой возрастной группы, от этого в нашей таблице отсутствует низший класс, что в целом походит на реальность, т.к. клиенты с теми запросами, которые мы вывели при категоризации и с низким доходом не обращались бы в банк для заёма.

На приведённой таблице мы видим что уровень дохода напрямую не влияет на результат и значения остаются в пределах 10% в любом варианте

- Как разные цели кредита влияют на его возврат в срок?

In [184]:
purpose_category_debt_dict=data[['purpose_category','debt', 'family_status_id']] # выделим необходимые столбцы для анализа

In [185]:
purpose_category_debt_result = pd.pivot_table(purpose_category_debt_dict, index=["purpose_category"], columns='debt', values='family_status_id', aggfunc='count') # группируем данные для анализа

In [186]:
purpose_category_debt_result

debt,0,1
purpose_category,,
автомобиль,3903,403
недвижимость,10029,782
образование,3643,370
свадьба,2138,186


In [187]:
print('процент клиентов чья цель покупки автомобиль имели задолженость: {0:.2%}'.format(purpose_category_debt_result.loc['автомобиль', 1] / 
                                                                                       purpose_category_debt_result.loc['автомобиль'].sum()))
print('процент клиентов чья цель покупки недвижимость имели задолженость: {0:.2%}'.format(purpose_category_debt_result.loc['недвижимость', 1] / 
                                                                                       purpose_category_debt_result.loc['недвижимость'].sum()))
print('процент клиентов чья цель покупки образование имели задолженость: {0:.2%}'.format(purpose_category_debt_result.loc['образование', 1] / 
                                                                                       purpose_category_debt_result.loc['образование'].sum()))
print('процент клиентов чья цель покупки свадьба имели задолженость: {0:.2%}'.format(purpose_category_debt_result.loc['свадьба', 1] / 
                                                                                       purpose_category_debt_result.loc['свадьба'].sum()))

процент клиентов чья цель покупки автомобиль имели задолженость: 9.36%
процент клиентов чья цель покупки недвижимость имели задолженость: 7.23%
процент клиентов чья цель покупки образование имели задолженость: 9.22%
процент клиентов чья цель покупки свадьба имели задолженость: 8.00%


**Вывод**

Данные показывают что самыми платежеспособными в данной метрике были клиенты обращающиеся в банк для покупки недвижимости и проведения свадьбы

##  Общий вывод

# Вывод по работе с данными

В данных было достаточно много грубых технических ошибок нуждающихся в предобработке. Данные ошибки влияли на результаты исследования и для точных расчётов нужно передать информацию специалисту по базе данных для устранения проблем.

# Вывод аналитического исследования

В ходе исследования было установлено что самыми благонадёжными клиентами являются те, чьи цели была операция с недвижимостью: 7.23%.
В свою очередь самой неплатежеспособной группой оказались те, клиенты что обращались с целью покупки автомобиля: 9.36%